In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Google A2A (Agent-to-Agent) Communication

This notebook introduces you to Google's Agent-to-Agent (A2A) protocol, a standardized way for AI agents to communicate and collaborate.  

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/a2aproject/a2a-samples/blob/main/notebooks/a2a_quickstart.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fa2aproject%2Fa2a-samples%2Fmain%2Fnotebooks%2Fa2a_quickstart.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/a2aproject/a2a-samples/main/notebooks/a2a_quickstart.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/a2aproject/a2a-samples/blob/main/notebooks/a2a_quickstart.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/a2aproject/a2a-samples/blob/main/notebooks/a2a_quickstart.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## What You'll Build

A three-agent system that works together to analyze trending topics:
1. **Trending Topics Agent** - Searches the web for current trending topics
2. **Trend Analyzer Agent** - Performs deep analysis with quantitative data
3. **Host Agent** - Orchestrates the other agents to provide comprehensive insights

<img src="https://storage.googleapis.com/github-repo/a2a/a2a-diagram.png" alt="drawing" width="1000"/>

## Prerequisites

- Python 3.11+
- Google Cloud Project with Vertex AI enabled
- Basic understanding of async Python

## Other Resources

- [Google ADK Documentation](https://google.github.io/adk-docs/)
- [A2A Protocol Specification](https://github.com/google/a2a)
- [Vertex AI Documentation](https://cloud.google.com/vertex-ai)
- Codelabs:
  - [Google's Agent Stack in Action: ADK, A2A, MCP on Google Cloud](https://codelabs.developers.google.com/instavibe-adk-multi-agents/instructions)
  - [Getting Started with Agent-to-Agent (A2A) Protocol: Gemini on Cloud Run](https://codelabs.developers.google.com/intro-a2a-purchasing-concierge)
  - [Getting Started with MCP, ADK and A2A](https://codelabs.developers.google.com/codelabs/currency-agent)

#### Important!
A2A is a work in progress (WIP) thus, in the near future there might be changes that are different from what demonstrated in this code.

### Setup and Installation

First, let's install the required dependencies:

In [1]:
# Install required packages
%pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0 python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## 1. Introduction to A2A

### What is Agent-to-Agent (A2A) Communication?

A2A is a standardized protocol that enables AI agents to:
- **Discover** each other's capabilities
- **Communicate** using a common JSON-RPC based protocol
- **Collaborate** to solve complex tasks
- **Stream** responses for real-time interactions

### Environment Configuration

In [2]:
# Targeted workaround for google-adk==1.9.0 compatibility with a2a-sdk==0.3.0
# This cell shall be removed when google-adk releases the version next to >1.9.0
# (after https://github.com/google/adk-python/pull/2297)


import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver


class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [3]:
import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any

import httpx
import nest_asyncio
import uvicorn

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

In [ ]:
# # Set Google Cloud Configuration
# os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'
# os.environ['GOOGLE_CLOUD_PROJECT'] = (
#     '[your-project-id]'  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# )
# os.environ['GOOGLE_CLOUD_LOCATION'] = (
#     'us-central1'  # Replace with your location
# )

# load_dotenv()

# print('Environment variables configured:')
# print(f'GOOGLE_GENAI_USE_VERTEXAI: {os.environ["GOOGLE_GENAI_USE_VERTEXAI"]}')
# print(f'GOOGLE_CLOUD_PROJECT: {os.environ["GOOGLE_CLOUD_PROJECT"]}')
# print(f'GOOGLE_CLOUD_LOCATION: {os.environ["GOOGLE_CLOUD_LOCATION"]}')

In [ ]:
# # Authenticate your notebook environment (Colab only)
# if 'google.colab' in sys.modules:
#     from google.colab import auth

#     auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

In [4]:
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [5]:
# Setup logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
)

## 2. Building Your A2A System

Let's build our three-agent system step by step. We'll create:

1. **Trending Topics Agent** - Finds current trending topics
2. **Trend Analyzer Agent** - Analyzes trends with quantitative data
3. **Host Agent** - Orchestrates the other agents (sequentially)

### Agent 1: Trending Topics Agent

This agent searches the web for trending topics and returns a list of current trends.

In [31]:
Agent

google.adk.agents.llm_agent.LlmAgent

In [6]:
# Create the Trending Topics ADK Agent
trending_agent = Agent(
    model='gemini-2.5-flash',
    name='trending_topics_agent',
    instruction="""
    You are a social media trends analyst. Your job is to search the web for current trending topics,
    particularly from social platforms.

    When asked about trends:
    1. Search for "trending topics today" or similar queries
    2. Extract the top 3 trending topics
    3. Return them in a JSON format

    Focus on current, real-time trends from the last 24 hours.

    You MUST return your response in the following JSON format:
    {
        "trends": [
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            },
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            },
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            }
        ]
    }

    Only return the JSON object, no additional text.
    """,
    tools=[google_search],
)

print('Trending Topics Agent created successfully!')

Trending Topics Agent created successfully!


In [9]:
AgentCard

a2a.types.AgentCard

In [10]:
trending_agent_card = AgentCard(
    name='Trending Topics Agent',
    url='http://localhost:10020',
    description='Searches the web for current trending topics from social media',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='find_trends',
            name='Find Trending Topics',
            description='Searches for current trending topics on social media',
            tags=['trends', 'social media', 'twitter', 'current events'],
            examples=[
                "What's trending today?",
                'Show me current Twitter trends',
                'What are people talking about on social media?',
            ],
        )
    ],
)

In [11]:
RemoteA2aAgent

google.adk.agents.remote_a2a_agent.RemoteA2aAgent

In [12]:
remote_trending_agent = RemoteA2aAgent(
    name='find_trends',
    description='Searches for current trending topics on social media',
    agent_card=f'http://localhost:10020{AGENT_CARD_WELL_KNOWN_PATH}',
)

/tmp/ipython-input-912843253.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_trending_agent = RemoteA2aAgent(


### Agent 2: Trend Analyzer Agent

This agent takes a specific trend and performs deep analysis with quantitative data.

In [13]:
# Create the Trend Analyzer ADK Agent
analyzer_agent = Agent(
    model='gemini-2.5-pro',
    name='trend_analyzer_agent',
    instruction="""
    You are a data analyst specializing in trend analysis. When given a trending topic,
    perform deep research to find quantitative data and insights.

    For each trend you analyze:
    1. Search for statistics, numbers, and metrics related to the trend
    2. Look for:
       - Engagement metrics (views, shares, mentions)
       - Growth rates and timeline
       - Geographic distribution
       - Related hashtags or keywords
    3. Provide concrete numbers and data points

    Keep it somehow concise

    Always prioritize quantitative information over qualitative descriptions.
    """,
    tools=[google_search],
)

print('Trend Analyzer Agent created successfully!')

Trend Analyzer Agent created successfully!


In [14]:
analyzer_agent_card = AgentCard(
    name='Trend Analyzer Agent',
    url='http://localhost:10021',
    description='Performs deep analysis of trends with quantitative data',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='analyze_trend',
            name='Analyze Trend',
            description='Provides quantitative analysis of a specific trend',
            tags=['analysis', 'data', 'metrics', 'statistics'],
            examples=[
                'Analyze the #ClimateChange trend',
                'Get metrics for the Taylor Swift trend',
                'Provide data analysis for AI adoption trend',
            ],
        )
    ],
)

In [15]:
remote_analyzer_agent = RemoteA2aAgent(
    name='analyze_trend',
    description='Provides quantitative analysis of a specific trend',
    agent_card=f'http://localhost:10021{AGENT_CARD_WELL_KNOWN_PATH}',
)

/tmp/ipython-input-738464179.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_analyzer_agent = RemoteA2aAgent(


### Agent 3: Host Agent (Orchestrator)

The Host Agent coordinates between the other two agents to provide comprehensive trend analysis.

In [17]:
SequentialAgent

google.adk.agents.sequential_agent.SequentialAgent

In [16]:
# Create the Host ADK Agent
host_agent = SequentialAgent(
    name='trend_analysis_host',
    sub_agents=[remote_trending_agent, remote_analyzer_agent],
)

In [18]:
host_agent_card = AgentCard(
    name='Trend Analysis Host',
    url='http://localhost:10022',
    description='Orchestrates, sequentially, trend discovery and analysis using specialized agents',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='comprehensive_trend_analysis',
            name='Comprehensive Trend Analysis',
            description='Finds trending topics and provides deep analysis of the most relevant one',
            tags=['trends', 'analysis', 'orchestration', 'insights'],
            examples=[
                'Analyze current trends',
                "What's trending and why is it important?",
                'Give me a comprehensive trend report',
            ],
        )
    ],
)

## 3. Running

Now let's put everything together. We'll create helper functions to start our agents and run the complete system.

### Starting the A2A Servers

Create function to run each agent as an A2A server:

In [36]:
A2AStarletteApplication

a2a.server.apps.jsonrpc.starlette_app.A2AStarletteApplication

In [21]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent.

    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card

    Returns:
        A2AStarletteApplication instance
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [22]:
# Apply nest_asyncio
nest_asyncio.apply()

# Store server tasks
server_tasks: list[asyncio.Task] = []


async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',  # Important: let uvicorn use the current loop
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """Start all servers in the same event loop."""
    # Create tasks for all servers
    tasks = [
        asyncio.create_task(
            run_agent_server(trending_agent, trending_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(analyzer_agent, analyzer_agent_card, 10021)
        ),
        asyncio.create_task(
            run_agent_server(host_agent, host_agent_card, 10022)
        ),
    ]

    # Give servers time to start
    await asyncio.sleep(2)

    print('✅ All agent servers started!')
    print('   - Trending Agent: http://127.0.0.1:10020')
    print('   - Analyzer Agent: http://127.0.0.1:10021')
    print('   - Host Agent: http://127.0.0.1:10022')

    # Keep servers running
    try:
        await asyncio.gather(*tasks)
    except KeyboardInterrupt:
        print('Shutting down servers...')


# Run in a background thread


def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


# Start the thread
server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

# Wait for servers to be ready
time.sleep(3)

/tmp/ipython-input-2220589873.py:19: UserWarning: [EXPERIMENTAL] A2aAgentExecutorConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  config = A2aAgentExecutorConfig()
/tmp/ipython-input-2220589873.py:20: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  executor = A2aAgentExecutor(runner=runner, config=config)


✅ All agent servers started!
   - Trending Agent: http://127.0.0.1:10020
   - Analyzer Agent: http://127.0.0.1:10021
   - Host Agent: http://127.0.0.1:10022


## 4. Testing the System

### Call the A2A agents (the 2 remote agents, and the host agent that refers to the 2 remote agents as sub agents)

In [23]:
class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # Create the message object
            message_obj = create_text_message_object(content=message)

            # Send the message and collect responses
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # The response is a tuple - get the first element (Task object)
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # First element of the tuple

                # Extract text: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return 'No response received'

In [24]:
a2a_client = A2ASimpleClient()

In [25]:
async def test_trending_topics() -> None:
    """Test trending topics agent."""
    trending_topics = await a2a_client.create_task(
        'http://localhost:10020', "What's trending today?"
    )
    print(trending_topics)


# Run the async function
asyncio.run(test_trending_topics())

/usr/local/lib/python3.12/dist-packages/google/adk/a2a/executor/a2a_agent_executor.py:184: UserWarning: [EXPERIMENTAL] convert_a2a_request_to_adk_run_args: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  run_args = convert_a2a_request_to_adk_run_args(context)
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/converters/request_converter.py:64: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  convert_a2a_part_to_genai_part(part)
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/executor/a2a_agent_executor.py:214: UserWarning: [EXPERIMENTAL] TaskResultAggregator: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  task_result_aggregator = TaskResu

```json
{
    "trends": [
        {
            "topic": "NBA Player Buzz",
            "description": "Discussions around NBA players like Tyrese Maxey, Victor Wembanyama, and Joel Embiid, along with teams like the Clippers, are highly active on social media platforms, particularly X (formerly Twitter).",
            "reason": "The NBA season is currently underway, leading to continuous fan engagement, game highlights, player performances, and team-related news dominating conversations on social platforms."
        },
        {
            "topic": "Taylor Swift's New Album Anticipation",
            "description": "There is significant buzz and excitement surrounding Taylor Swift's new album, which is set to be released on Friday. Fans are actively discussing and preparing for the launch.",
            "reason": "Taylor Swift has a massive and highly engaged fanbase ('Swifties') who generate immense anticipation and social media activity around her new music releases, making it a top

/usr/local/lib/python3.12/dist-packages/google/adk/a2a/executor/a2a_agent_executor.py:216: UserWarning: [EXPERIMENTAL] convert_event_to_a2a_events: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  for a2a_event in convert_event_to_a2a_events(
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/converters/event_converter.py:512: UserWarning: [EXPERIMENTAL] convert_event_to_a2a_message: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  message = convert_event_to_a2a_message(event, invocation_context)
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/converters/event_converter.py:350: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  a2a_part = conve

In [26]:
async def test_analysis() -> None:
    """Test analysis agent."""
    analysis = await a2a_client.create_task(
        'http://localhost:10021', 'Analyze the trend AI in Social Media'
    )
    print(analysis)


# Run the async function
asyncio.run(test_analysis())

## AI's Proliferation Across Social Media Platforms: A Quantitative Analysis

Artificial intelligence is rapidly becoming the backbone of the social media landscape, with its market size in the sector projected to surge from approximately USD 2.2-2.96 billion in 2024 to between USD 10.3 billion and USD 54.07 billion by the early 2030s. This exponential growth is underpinned by AI's deep integration into content creation, user engagement, and marketing strategies.

**Content and Engagement Metrics:**

A staggering 80% of content recommendations on social media platforms are now powered by AI algorithms, a key factor in boosting user retention rates. The creation of content itself is also heavily influenced by AI, with an estimated 71% of images shared on social media being AI-generated. This has enabled marketers to significantly increase their output, with some reporting the ability to generate up to 72 posts per week using generative AI tools. Businesses that have adopted AI for their

In [27]:
async def test_host_analysis() -> None:
    """Test host analysis agent."""
    host_analysis = await a2a_client.create_task(
        'http://localhost:10022',
        'Find the most relevant trends in the web today, choose randomly one of the top '
        'trends, and give me a complete analysis of it with quantitative data',
    )
    print(host_analysis)


# Run the async function
asyncio.run(test_host_analysis())

/usr/local/lib/python3.12/dist-packages/google/adk/agents/remote_a2a_agent.py:358: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  converted_part = convert_genai_part_to_a2a_part(part)
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/converters/event_converter.py:206: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  return convert_a2a_message_to_event(message, author, invocation_context)
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/converters/event_converter.py:272: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  part =

### Tale of Two Phenoms and ailing Star Dominates NBA's Opening Buzz

A quantitative analysis of the NBA's current social media landscape reveals a trend driven by divergent narratives: the record-shattering performance of San Antonio's Victor Wembanyama, the explosive scoring of Philadelphia's Tyrese Maxey, and widespread concern over the health of 76ers' star Joel Embiid.

#### Victor Wembanyama: Unprecedented Performance Fuels Global Hype

The French phenom's season-opening performance has generated staggering social media engagement, backed by historic on-court statistics.

*   **Record-Breaking Stat Line:** In a 125-92 victory over the Dallas Mavericks, Wembanyama recorded 40 points, 15 rebounds, and 3 blocked shots in just 30 minutes of play. This performance set a new Spurs franchise record for most points in a season opener. He is the first player since 1977-78 to post a stat line of 40+ points, 15+ rebounds, and 0 turnovers while shooting over 70% from the field.

*   **Massiv

/usr/local/lib/python3.12/dist-packages/google/adk/a2a/executor/a2a_agent_executor.py:216: UserWarning: [EXPERIMENTAL] convert_event_to_a2a_events: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  for a2a_event in convert_event_to_a2a_events(
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/converters/event_converter.py:512: UserWarning: [EXPERIMENTAL] convert_event_to_a2a_message: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  message = convert_event_to_a2a_message(event, invocation_context)
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/converters/event_converter.py:350: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  a2a_part = conve

## Summary

Congratulations! You've successfully built a multi-agent system using Google's A2A protocol. Here's what you've learned:

1. **A2A Protocol Basics**: How agents discover and communicate with each other
2. **ADK Integration**: Creating ADK agents and wrapping them for A2A
3. **Agent Orchestration**: Building a Host Agent that coordinates multiple agents
4. **Practical Implementation**: Running and testing a complete multi-agent system

### Next Steps

- **Deploy Your Agents**: Deploy agents to Cloud Run or other platforms
- **Add Authentication**: Implement security for production use
- **Create More Agents**: Build agents for your specific use cases, even using other frameworks
- **Advanced Patterns**: Explore agent chains, parallel execution, and more
- **Callbacks**: Add in the Google ADK agents the before and after callbacks of the agent, model and tool, to increase observability

Happy agent building! 🚀

# Appendix

### Why Use Google A2A (Agent-to-Agent) Protocol

Google's Agent-to-Agent (A2A) protocol is a standardized communication framework that enables AI agents to discover, communicate, and collaborate with each other using a common JSON-RPC based protocol.  
It provides a uniform way for agents to interact, regardless of their underlying implementation.  

#### 1. Standardized Communication Protocol

- A2A provides a consistent, JSON-RPC based protocol that any agent can implement
- Agents can communicate without needing to know each other's internal implementation details
- The protocol supports streaming responses for real-time interactions

#### 2. Agent Discovery and Metadata

- Agents expose their capabilities through standardized metadata (AgentCard)
- Each agent publishes its skills, input/output modes, and capabilities
- Host agents can dynamically discover what other agents can do through the `.well-known/agent-card.json` endpoint

#### 3. Orchestration and Composition

- Enables building complex multi-agent systems where a host agent can orchestrate multiple specialized agents
- Supports sequential and parallel task execution patterns
- Allows for sophisticated agent collaboration workflows

#### 4. Platform Independence

- A2A servers can wrap agents from different frameworks (not just ADK)
- Agents can be deployed as independent services on different infrastructure
- Promotes loose coupling between agents

### Differences: Using ADK Agents Directly vs. Through A2A

#### Using ADK Agents Directly

```python
# Conceptual Example: Defining Hierarchy
from google.adk.agents import LlmAgent, BaseAgent

# Define individual agents
greeter = LlmAgent(name="Greeter", model="gemini-2.5-pro")
task_doer = BaseAgent(name="TaskExecutor") # Custom non-LLM agent

# Create parent agent and assign children via sub_agents
coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.5-pro",
    description="I coordinate greetings and tasks.",
    sub_agents=[ # Assign sub_agents here
        greeter,
        task_doer
    ]
)
```

__Use Direct ADK for Multi-Agents System When:__

- All agents are tightly related and always used together
- Google ADK is the framework choice, and simplicity is prioritized
- Performance of in-process communication is critical
- You don't need distributed deployment
- No built-in service discovery is needed

#### Using ADK Agents Through A2A

__Use A2A for Multi-Agents System When:__

- Building complex multi-agent systems
- Agents need to be developed, deployed, and scaled independently
- You want to integrate agents from different teams or frameworks
- You need dynamic agent discovery and composition
- Building a platform where agents can be added/removed dynamically
- You want to enable third-party agent integration